## Anime search engine based description using SBERT and FAISS

We want to make anime search engine based description by user input using SBERT and FAISS, for example if user give query input "Anime romance about daily life in school between childhood friend", the search engine will give output some top anime name with the most similarity to the query input

### 1. Dataset

for dataset we will use anime with review dataset from kaggle, you can acces it here https://www.kaggle.com/datasets/marlesson/myanimelist-dataset-animes-profiles-reviews

In [1]:
import pandas as pd

# load dataset (edit the file path according your dataset path)
anime_data = pd.read_csv('dataset/raw/animes.csv')
review_data = pd.read_csv('dataset/raw/reviews.csv')

We use that dataset because we want to take more information about the anime for training our model. So instead just using basic information about the anime like synopsis, genre, year aired, etc. We will also using review user from each anime for improving our model

#### 2. EDA (Exploratory Data Analysys)

In [2]:
anime_data.sample(5)

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
887,30902,Yuru Yuri Nachuyachumi!+,Extra episodes of Yuru Yuri Nachuyachumi!,"['Slice of Life', 'Comedy', 'Shoujo Ai']","Aug 21, 2015 to Sep 18, 2015",2.0,38033,2263,852.0,7.84,https://cdn.myanimelist.net/images/anime/2/759...,https://myanimelist.net/anime/30902/Yuru_Yuri_...
15386,7712,Kansen 2: Inzai Toshi,Kansen 2 (like the first episode) takes place ...,['Hentai'],"Feb 26, 2010 to Jun 25, 2010",2.0,4222,6145,NaN,6.45,https://cdn.myanimelist.net/images/anime/2/249...,https://myanimelist.net/anime/7712/Kansen_2__I...
7309,32581,Gaza: Changing the World,A promotional video produced by the Shizuoka M...,"['Drama', 'Supernatural']","Jun 27, 2015",1.0,233,12870,9750.0,5.55,https://cdn.myanimelist.net/images/anime/10/78...,https://myanimelist.net/anime/32581/Gaza__Chan...
16522,9935,Chocolat no Mahou,"The ""bittersweet, mysterious story"" revolves a...","['Drama', 'Magic', 'Shoujo', 'Supernatural']","Mar 3, 2011 to Jan 19, 2013",13.0,5466,5543,7175.0,6.26,https://cdn.myanimelist.net/images/anime/6/356...,https://myanimelist.net/anime/9935/Chocolat_no...
3393,22567,Osaru no Monkichi no Kin no Ono Gin no Ono,Monkichi the Monkey recreates the tale of The...,['Kids'],"Sep 21, 2001",1.0,178,13801,8699.0,5.89,https://cdn.myanimelist.net/images/anime/6/588...,https://myanimelist.net/anime/22567/Osaru_no_M...


In [3]:
anime_data.describe()

,uid,episodes,members,popularity,ranked,score
count,19311.000000,18605.000000,1.931100e+04,19311.000000,16099.000000,18732.000000
mean,19358.904096,11.460414,3.472609e+04,7720.830304,6866.524194,6.436107
std,14271.446515,47.950386,1.121772e+05,4676.786104,4390.018768,1.007941
min,1.000000,1.000000,2.500000e+01,1.000000,1.000000,1.250000
25%,4833.500000,1.000000,3.880000e+02,3725.000000,2895.500000,5.770000
50%,18327.000000,2.000000,2.389000e+03,7539.000000,6963.000000,6.410000
75%,33896.500000,12.000000,1.450150e+04,11613.000000,10601.500000,7.150000
max,40960.000000,3057.000000,1.871043e+06,16338.000000,14675.000000,9.230000


In [4]:
anime_data.describe(include='O')

,title,synopsis,genre,aired,img_url,link
count,19311,18336,19311,19311,19131,19311
unique,16214,15194,4851,11127,16051,16216
top,Hakkenden: Touhou Hakken Ibun 2nd Season,No synopsis has been added for this series yet...,['Hentai'],Not available,https://cdn.myanimelist.net/images/anime/1071/...,https://myanimelist.net/anime/18055/Hakkenden_...
freq,3,46,1837,372,3,3


In [5]:
anime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19311 entries, 0 to 19310
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   uid         19311 non-null  int64  
 1   title       19311 non-null  object 
 2   synopsis    18336 non-null  object 
 3   genre       19311 non-null  object 
 4   aired       19311 non-null  object 
 5   episodes    18605 non-null  float64
 6   members     19311 non-null  int64  
 7   popularity  19311 non-null  int64  
 8   ranked      16099 non-null  float64
 9   score       18732 non-null  float64
 10  img_url     19131 non-null  object 
 11  link        19311 non-null  object 
dtypes: float64(3), int64(3), object(6)
memory usage: 1.8+ MB


In [6]:
review_data.sample(5)

,uid,profile,anime_uid,text,score,scores,link
132502,222698,LilMsPocketWeeb,4224,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '8', 'Animation': '1...",https://myanimelist.net/reviews.php?id=222698
99915,297809,underscore21,37349,\n \n \n \n ...,6,"{'Overall': '6', 'Story': '7', 'Animation': '4...",https://myanimelist.net/reviews.php?id=297809
93006,44,VyseN,857,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '8', 'Animation': '9...",https://myanimelist.net/reviews.php?id=44
39867,190948,The_Languid_One,205,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '8', 'Animation': '9...",https://myanimelist.net/reviews.php?id=190948
173938,19096,HalRyder,4896,\n \n \n \n ...,6,"{'Overall': '6', 'Story': '7', 'Animation': '2...",https://myanimelist.net/reviews.php?id=19096


In [7]:
review_data.describe()

,uid,anime_uid,score
count,192112.000000,192112.000000,192112.000000
mean,187648.127525,15273.300283,7.570235
std,98748.902397,13480.565379,2.255167
min,1.000000,1.000000,0.000000
25%,101779.500000,2167.000000,6.000000
50%,210913.500000,10793.000000,8.000000
75%,270383.000000,30205.000000,9.000000
max,325747.000000,40807.000000,11.000000


In [8]:
review_data.describe(include='O')

,profile,text,scores,link
count,192112,192112,192112,192112
unique,47885,130440,29837,130519
top,Sidewinder51,\n \n \n \n ...,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=321183
freq,762,8,13900,4


In [9]:
review_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192112 entries, 0 to 192111
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   uid        192112 non-null  int64 
 1   profile    192112 non-null  object
 2   anime_uid  192112 non-null  int64 
 3   text       192112 non-null  object
 4   score      192112 non-null  int64 
 5   scores     192112 non-null  object
 6   link       192112 non-null  object
dtypes: int64(3), object(4)
memory usage: 10.3+ MB


In [10]:
review_data['text'].iloc[1]

"\n           \n         \n           \n             \n           \n         \n         \n           more pics \n         \n       \n         \n       \n         \n           Overall \n           10 \n         \n         \n           Story \n           10 \n         \n                   \n             Animation \n             10 \n           \n           \n             Sound \n             10 \n           \n                 \n           Character \n           10 \n         \n         \n           Enjoyment \n           10 \n         \n       \n     \n\n                    \n    Let me start off by saying that Made in Abyss is a series that deserves to be bestowed with the title of a Masterpiece of an anime series. This may sound like I'm placing the series on a pedestal, and you could say that personal bias has influenced by decision to rate MIA a full 10/10 for all the components, but please allow me to try my best to break down and pinpoint just exactly why it has earned such a place

#### 3. Preprocessing

Before using our dataset to our model, we need to do preprocessing to dataset. as you can see in exploratory, our dataset its still so messy.

In [11]:
"""Filtering anime based on many reviews of each anime 
(Because i dont want my Computer to explode when training too many datas :) ), 
but you can skip this code if you're confident with your computer capabilities"""

selected_reviews = review_data[['anime_uid', 'text']].groupby(by=['anime_uid']).count().sort_values('text', ascending=False)
selected_reviews = selected_reviews[selected_reviews.text > 15].reset_index()

# Adding all reviews/sum up all reviews from the same anime

reviews_joined = review_data[['anime_uid', 'text']][review_data['anime_uid'].isin(selected_reviews['anime_uid'])]
reviews_joined = reviews_joined.groupby(by=['anime_uid']).sum().reset_index()

In [12]:
reviews_joined.sample(5)

,anime_uid,text
335,1285,\n \n \n \n ...
133,256,\n \n \n \n ...
1438,31804,\n \n \n \n ...
1163,21843,\n \n \n \n ...
1126,20689,\n \n \n \n ...


In [13]:
"""Joining our 2 dataset, anime and review, so we have 1 dataframe contains basic information 
sum up review from each anime"""

anime_data = anime_data.drop_duplicates(subset=['uid'])
reviews_joined.rename(columns={'anime_uid' : 'uid', 'text' : 'review'}, inplace=True)
animes_reviews = pd.merge(anime_data, reviews_joined, how='right', on=['uid']).drop_duplicates()

# we will only take columns that we think have the potential to be used as descriptions

animes_reviews = animes_reviews.drop(['members', 'popularity', 'ranked', 'img_url', 'link'],axis=1)
animes_reviews = animes_reviews.dropna()

In [14]:
"""We need cleaning or text dataset, so each column can be a description from each anime.
Like cleanng unusual punctuation and add / change some column to string/object"""

def preProcessReview(row):
    res = str(row['review'])
    
    while '\n' in res or '\r' in res or "\'" in res or '  ' in res:
        res = res.replace('\n','')
        res = res.replace('\r','')
        res = res.replace("\'",'')
        res = res.replace('  ',' ')
    
    return res

def preProcessScore(row):
    res = str(row['score'])
    
    return 'Score ' + res

def preProcessEpisodes(row):
    res = str(row['episodes'])
    res = res.replace(".0",'')
    
    return 'with ' + res + ' Episodes'

def preProcessGenre(row):
    res = row['genre']
    res = res.replace("'",'')
    res = res.replace('[','')
    res = res.replace(']','')
    
    return 'Genre ' + res

def preProcessSynopsis(row):
    res = str(row['synopsis'])
    while '\n' in res or '\r' in res or "\'" in res or '  ' in res:
        res = res.replace('\n','')
        res = res.replace('\r','')
        res = res.replace("\'",'')
        res = res.replace('  ',' ')
        
    return 'Synopsis, ' + res

animes_reviews['synopsis'] = animes_reviews.apply(preProcessSynopsis, axis=1)
animes_reviews['genre'] = animes_reviews.apply(preProcessGenre, axis=1)
animes_reviews['episodes'] = animes_reviews.apply(preProcessEpisodes, axis=1)
animes_reviews['score'] = animes_reviews.apply(preProcessScore, axis=1)
animes_reviews['review'] = animes_reviews.apply(preProcessReview, axis=1)

In [15]:
"""Because our goal is to make search engine based description, 
we need to sum up all column from each anime into 1 text description for that anime"""

animes_reviews['text'] = animes_reviews[animes_reviews.columns[1:]].apply(
    lambda x: ', '.join(x.dropna().astype(str)),
    axis=1
)

"""filtering data into the full description text and title as identifier.
after that we need slicing the description string
(Because i dont want my Computer to explode when training too many datas :) ).
of course you can take as much as you want"""

def sliceString(row):
    res = str(row['text'])
    
    return res[:5000]

data = animes_reviews[['title','text']].copy()
data['text'] = data.apply(sliceString, axis=1)

In [16]:
data.sample(5)

,title,text
734,Kami nomi zo Shiru Sekai,"Kami nomi zo Shiru Sekai, Synopsis, Keima Kats..."
175,Blood: The Last Vampire,"Blood: The Last Vampire, Synopsis, Teropterids..."
1044,Inferno Cop,"Inferno Cop, Synopsis, Jack Knife Edge Town is..."
1738,Ninja Batman,"Ninja Batman, Synopsis, Batman Ninja takes a j..."
1369,Makura no Danshi,"Makura no Danshi, Synopsis, Whispering sweet l..."


In [28]:
data['text'].iloc[1]

'Cowboy Bebop: Tengoku no Tobira, Synopsis, Another day, another bounty—such is the life of the often unlucky crew of the Bebop. However, this routine is interrupted when Faye, who is chasing a fairly worthless target on Mars, witnesses an oil tanker suddenly explode, causing mass hysteria. As casualties mount due to a strange disease spreading through the smoke from the blast, a whopping three hundred million woolong price is placed on the head of the supposed perpetrator. With lives at stake and a solution to their money problems in sight, the Bebop crew springs into action. Spike, Jet, Faye, and Edward, followed closely by Ein, split up to pursue different leads across Alba City. Through their individual investigations, they discover a cover-up scheme involving a pharmaceutical company, revealing a plot that reaches much further than the ragtag team of bounty hunters could have realized. [Written by MAL Rewrite], Genre Action, Drama, Mystery, Sci-Fi, Space, Sep 1, 2001, with 1 Episo

In [17]:
""" Optional: we can make our processed data into csv,
so we dont need to run the preprocessing code again when run our model codes"""

data.to_csv('dataset/processed.csv', sep='/', index=False)

#### 4. Model

In [18]:
# Optional: if you save your processed dataset into csv

import pandas as pd

data = pd.read_csv('dataset/processed.csv',sep='/')

In [19]:
"""SBERT
SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings.
You can use this framework to compute sentence / text embeddings for more than 100 languages
These embeddings can then be compared e.g. with cosine-similarity to find sentences with a similar meaning

You can read it more in https://www.sbert.net
"""

# We are using pretrained model from SBERT 
# You can search it in https://www.sbert.net/docs/pretrained_models.html

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

c:\Users\KEY\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
"""FAISS
Faiss is a library for efficient similarity search and clustering of dense vectors. 
It contains algorithms that search in sets of vectors of any size, 
up to ones that possibly do not fit in RAM. 
It also contains supporting code for evaluation and parameter tuning.

Faiss is written in C++ with complete wrappers for Python. 
Some of the most useful algorithms are implemented on the GPU. 
It is developed by Facebook AI Research.

you can read it more in  https://faiss.ai
"""

# We encode our text description anime with our model
# and then we mapping our anime id and encoded description
# into 768 dimentional dense matrix

import faiss
import numpy as np
encoded_data = model.encode(data.text.tolist())
encoded_data = np.asarray(encoded_data.astype('float32'))
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.array(range(0, len(data))))
faiss.write_index(index, 'animes.index')

# Optional: save model to directory
model.save('model')

In [21]:
# define function to return top_k anime dictionary based query

def fetch_movie_info(dataframe_idx):
    info = data.iloc[dataframe_idx]
    meta_dict = dict()
    meta_dict['title'] = info['title']
    return meta_dict
    
def search(query, top_k, index, model):
    query_vector = model.encode([query])
    top_k = index.search(query_vector, top_k)
    top_k_ids = top_k[1].tolist()[0]
    top_k_ids = list(np.unique(top_k_ids))
    results =  [fetch_movie_info(idx) for idx in top_k_ids]
    return results

In [30]:
# change the query text, an then run to get anime recomendation

query="Anime about food and cooking"
results=search(query, top_k=10, index=index, model=model)
print("\n")
for result in results:
    print('\t',result)



	 {'title': 'Yakitate!! Japan'}
	 {'title': 'Bakuretsu Tenshi'}
	 {'title': 'Ben-To'}
	 {'title': 'Koufuku Graffiti'}
	 {'title': 'Shokugeki no Souma'}
	 {'title': 'Pan de Peace!'}
	 {'title': 'Amaama to Inazuma'}
	 {'title': 'Isekai Shokudou'}
	 {'title': 'Ramen Daisuki Koizumi-san'}
	 {'title': 'Emiya-san Chi no Kyou no Gohan'}
